# Library Implementation

Cài đặt và import các thư viện cần thiết cho pipeline huấn luyện CrossEncoder với hard negative mining, gồm:

* underthesea: xử lý tiếng Việt

* sentence-transformers: lấy embedding + CrossEncoder

* faiss-cpu: tìm kiếm vector nhanh

* sklearn: đánh giá bằng các metrics cơ bản

* torch, datasets, pandas, tqdm: huấn luyện và xử lý dữ liệu

In [1]:
%%capture
!pip install -qU sentence-transformers
!pip install -q faiss-cpu
!pip install -qU underthesea
import json
import random
import numpy as np

from pprint import pprint
from sentence_transformers import CrossEncoder, InputExample, util
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
from sklearn.model_selection import KFold

import torch, gc
import os

Bắt buộc phải dùng bản mới nhất của sentence-transformers để phải có các hàm sau:
* CrossEncoderModelCardData
* CrossEncoderTrainer
* CrossEncoderTrainingArguments

In [2]:
print("Phiên bản của Sentence-transformers")
!pip show sentence-transformers
import sentence_transformers as ce
print("\nCác hàm của cross encoder")
pprint(dir(ce.cross_encoder))

Phiên bản của Sentence-transformers
Name: sentence-transformers
Version: 5.1.1
Summary: Embeddings, Retrieval, and Reranking
Home-page: https://www.SBERT.net
Author: 
Author-email: Nils Reimers <info@nils-reimers.de>, Tom Aarsen <tom.aarsen@huggingface.co>
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, Pillow, scikit-learn, scipy, torch, tqdm, transformers, typing_extensions
Required-by: 

Các hàm của cross encoder
['CrossEncoder',
 'CrossEncoderModelCardData',
 'CrossEncoderTrainer',
 'CrossEncoderTrainingArguments',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'annotations',
 'data_collator',
 'fit_mixin',
 'losses',
 'model_card',
 'trainer',
 'training_args',
 'util']


Kiểm tra hiện có GPU hay không?

In [3]:
import torch
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Number of GPUs:", torch.cuda.device_count())
    print("Current GPU index:", torch.cuda.current_device())
    print("GPU name:", torch.cuda.get_device_name(torch.cuda.current_device()))

CUDA available: True
Number of GPUs: 1
Current GPU index: 0
GPU name: Tesla P100-PCIE-16GB


In [4]:
import logging
import traceback

import torch
from datasets import load_dataset

from sentence_transformers import (
    CrossEncoder,
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
)
from sentence_transformers.cross_encoder import (
    CrossEncoder,
    #CrossEncoderModelCardData,
    CrossEncoderTrainer,
    CrossEncoderTrainingArguments,
)
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import TripletEvaluator
from sentence_transformers.util import mine_hard_negatives
from sentence_transformers.cross_encoder.evaluation import CrossEncoderRerankingEvaluator
from underthesea import word_tokenize
from transformers import AutoTokenizer

from sentence_transformers.cross_encoder import losses
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

# Load a model

In [5]:
# =============================================
# I. Load a model to finetune with
# =============================================
#model = CrossEncoder('sentence-transformers/paraphrase-multilingual-mpnet-base-v2', num_labels=1)
model = CrossEncoder(
    "sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    max_length=512,  # quan trọng
    tokenizer_args={"truncation": True},
    num_labels=1
)
embedding_model = SentenceTransformer('/kaggle/input/stock-dataset-qibot/model/biencoder_bkai_epoch25_lr64_2e5')

print("Model max length:", model.max_length)
print("Model num labels:", model.num_labels)

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

Model max length: 512
Model num labels: 1


# Preprocessing data

In [6]:
# # =============================================
# # II. XỬ LÝ DỮ LIỆU (QA Dataset)
# # =============================================
# # 2a. Load the QA dataset: dataset.json
# logging.info("Read my QA training dataset")
# with open('/kaggle/input/stock-dataset-qibot/corpus.json', 'r',encoding ='utf-8') as f:
#     raw_data = json.load(f)
# print(f"Số mẫu ban đầu: {len(raw_data)}")
# raw_data[1]

In [ ]:
# from transformers import AutoTokenizer
# import matplotlib.pyplot as plt
# import numpy as np

# tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
# token_lengths= []
# for sample in raw_data:
#     tokens = tokenizer.encode_plus(sample['anchor'], sample['positive'], truncation=False)
#     token_lengths.append(len(tokens['input_ids']))

# # Thống kê cơ bản
# token_lengths_np = np.array(token_lengths)
# print(f"Tổng mẫu: {len(token_lengths)}")
# print(f"Min: {np.min(token_lengths_np)} tokens")
# print(f"Max: {np.max(token_lengths_np)} tokens")
# print(f"Trung bình: {np.mean(token_lengths_np):.2f} tokens")
# print(f"Median: {np.median(token_lengths_np)} tokens")
# print(f"% mẫu > 512 tokens: {(np.sum(token_lengths_np > 512) / len(token_lengths)) * 100:.2f}%")

# # Vẽ biểu đồ phân bố
# plt.hist(token_lengths_np, bins=50, color='skyblue', edgecolor='black')
# plt.axvline(x=512, color='red', linestyle='--', label='max_length = 512')
# plt.title("Phân bố độ dài tokens (anchor + positive)")
# plt.xlabel("Số tokens sau encode")
# plt.ylabel("Số lượng mẫu")
# plt.legend()
# plt.grid(True)
# plt.tight_layout()
# plt.show()

In [8]:
# # 2b. Tokenizing dataset
# for sample in raw_data:
#     sample['anchor'] = (word_tokenize(sample['question'], 'text'))
#     sample['positive'] = (word_tokenize(sample['answer'], 'text'))

# # 2c.Exclude invalid length samples
# def is_within_max_length(sample, tokenizer, model_max_length):
#     tokens = tokenizer.encode_plus(sample['anchor'], sample['positive'], truncation=False)
#     return len(tokens['input_ids']) <= model_max_length
    
# print(f"{torch.cuda.empty_cache()} {gc.collect()}")

# tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
# before_filter_len = len(raw_data)
# filtered_data = [sample for sample in raw_data if is_within_max_length(sample, tokenizer,model.max_length)]
# num_removed = before_filter_len - len(filtered_data)
# print(f" Đã loại bỏ {num_removed} mẫu vượt quá độ dài {model.max_length} tokens")
# # 2d. Exclude exact samples
# def deduplicate_by_text(data, keys=["anchor", "positive"]):
#     seen = set()
#     unique_data = []
#     for sample in data:
#         key = tuple(sample[k].strip().lower() for k in keys)
#         if key not in seen:
#             seen.add(key)
#             unique_data.append(sample)
#     return unique_data

# print(f"Trước khi loại trùng: {len(filtered_data)} mẫu")
# unique_data = deduplicate_by_text(filtered_data)
# print(f"\nSau khi loại trùng: {len(unique_data)} mẫu")

# # 2e. Spliting dataset into train, valid set, test_set
# output_file = 'dataset_processed.json'
# with open(output_file, 'w', encoding='utf-8') as f:
#     json.dump(unique_data, f, ensure_ascii=False, indent=2)

In [9]:
# with open('/kaggle/input/stock-dataset-qibot/dataset/finetune_crossEncoder/corpus_processed.json','r',encoding = 'utf-8') as f:
#     unique_data = json.load(f)
# len(unique_data)    

### Ví dụ Tokenization

In [ ]:
list = ['Theo Điều 211, Luật Chứng khoán 2019 và Nghị định 156/2020/NĐ-CP, hành vi thao túng giá chứng khoán bị nghiêm cấm và có thể bị xử phạt hành chính lên tới 3 tỷ đồng đối với cá nhân, 5 tỷ đồng với tổ chức. Trong trường hợp gây hậu quả nghiêm trọng, cá nhân vi phạm có thể bị truy cứu trách nhiệm hình sự với mức án tù đến 7 năm theo Điều 211 Bộ luật Hình sự. Các hành vi phổ biến bao gồm tạo cung – cầu giả, giao dịch khống, hoặc thông đồng giữa các bên để thao túng giá.',
       'So sánh giữa thị trường phái sinh và thị trường cơ sở về mục tiêu sử dụng và mức độ rủi ro.',
       'Thị trường cơ sở là nơi giao dịch các loại chứng khoán như cổ phiếu, trái phiếu... có tài sản thật, phản ánh giá trị nội tại của doanh nghiệp và chịu rủi ro biến động giá thông thường. Thị trường phái sinh bao gồm các công cụ như hợp đồng tương lai, được giao dịch dựa trên tài sản cơ sở, cho phép phòng ngừa rủi ro hoặc đầu cơ với đòn bẩy tài chính cao. Rủi ro thị trường phái sinh lớn hơn, yêu cầu nhà đầu tư có kiến thức chuyên sâu.',
       'Phân tích vai trò và giới hạn hoạt động của công ty quản lý quỹ trong thị trường chứng khoán Việt Nam.',
       'Công ty quản lý quỹ thực hiện việc đầu tư, quản lý tài sản cho các quỹ đầu tư và nhà đầu tư cá nhân. Theo Luật Chứng khoán 2019, công ty quản lý quỹ phải có vốn điều lệ tối thiểu 25 tỷ đồng, được cấp phép bởi UBCKNN và không được sử dụng tài sản quỹ để đảm bảo cho bên thứ ba. Vai trò chính là tối ưu hóa danh mục đầu tư theo mục tiêu lợi nhuận đã công bố, góp phần phát triển thị trường vốn chuyên nghiệp và minh bạch.']
for x in list:
    print('-'*80)
    print(word_tokenize(x,'text'))

--------------------------------------------------------------------------------
Theo Điều 211 , Luật_Chứng_khoán 2019 và Nghị_định 156 / 2020 / NĐ-CP , hành_vi thao_túng giá chứng_khoán bị nghiêm_cấm và có_thể bị xử_phạt hành_chính lên tới 3 tỷ đồng đối_với cá_nhân , 5 tỷ đồng với tổ_chức . Trong trường_hợp gây hậu_quả nghiêm_trọng , cá_nhân vi_phạm có_thể bị truy_cứu trách_nhiệm hình_sự với mức án tù đến 7 năm theo Điều 211 Bộ_luật_Hình_sự . Các hành_vi phổ_biến bao_gồm tạo cung – cầu giả , giao_dịch khống , hoặc thông_đồng giữa các bên để thao_túng giá .
--------------------------------------------------------------------------------
So_sánh giữa thị_trường phái_sinh và thị_trường cơ_sở về mục_tiêu sử_dụng và mức_độ rủi_ro .
--------------------------------------------------------------------------------
Thị_trường cơ_sở là nơi giao_dịch các loại chứng_khoán như cổ_phiếu , trái_phiếu ... có tài_sản thật , phản_ánh giá_trị nội_tại của doanh_nghiệp và chịu rủi_ro biến_động giá thông_t

In [24]:
# path ='/kaggle/input/stock-dataset-qibot/dataset/finetune_crossEncoder/corpus_processed.json'
# dataset = load_dataset("json", 
#                        data_files=path)['train'] # DatasetDict => Dataset
# print(f"\n Dataset đã load: {dataset}")
# df = dataset.to_pandas()
# print("\nPhân phối câu hỏi theo level:")
# print(df['level'].value_counts())

# Building hard Training set

In [11]:
# # =============================================
# ## III. Building hard Training set
# # =============================================
# # 3a. Generate hard dataset
# print(f"{torch.cuda.empty_cache()} {gc.collect()}")
# hard_dataset_random= mine_hard_negatives(
#     dataset=dataset,
#     model=embedding_model,
#     anchor_column_name="anchor",
#     positive_column_name="positive",
#     num_negatives=1, 
#     range_min=10,  
#     range_max=300,
#     max_score=0.8, 
#     margin = 0.05,
#     relative_margin=0.3, 
#     sampling_strategy="random", # top, random
#     batch_size=128, 
#     as_triplets=True,    # False nếu bạn dùng loss kiểu BCE
#     use_faiss=True, 
# )
# print(hard_dataset_random)
# pprint(hard_dataset_random[0])

In [12]:
# # 3a. Split hard dataset into train, validation set
# from datasets import Dataset, ClassLabel, DatasetDict
# import os


# def prepare_train_valid_test_split(hard_dataset, original_dataset, test_size_fixed=2000, val_ratio=0.1, seed=42):
#     import pandas as pd

#     # Bước 1: Merge dữ liệu để lấy cột `level`
#     df_hard = hard_dataset.to_pandas()
#     df_source = original_dataset.to_pandas()[["anchor", "level"]]
#     df_source_dedup = df_source.drop_duplicates(subset="anchor", keep="first")
#     df_merged = df_hard.merge(df_source_dedup, on="anchor", how="left")

#     # Bước 2: Encode `level` để stratify
#     unique_levels = sorted(set(original_dataset["level"]))
#     class_label = ClassLabel(names=[str(l) for l in unique_levels])
#     df_merged["level"] = df_merged["level"].astype(str)
#     #class_label = ClassLabel(names=["1", "2", "3"])
    
    
#     # Bước 3: Lấy tập test cố định 2000 mẫu theo stratified
#     from sklearn.model_selection import train_test_split
#     df_remaining, df_test = train_test_split(
#         df_merged,
#         test_size=test_size_fixed,
#         stratify=df_merged["level"],
#         random_state=seed,
#     )

#     # Bước 4: Chia remaining thành train và validation theo tỷ lệ 9:1
#     df_train, df_valid = train_test_split(
#         df_remaining,
#         test_size=val_ratio,
#         stratify=df_remaining["level"],
#         random_state=seed,
#     )

#     # Bước 5: Chuyển về Dataset + ép kiểu level
#     def convert(df):
#         df = df.reset_index(drop=True)
#         dset = Dataset.from_pandas(df)
#         dset = dset.map(lambda x: {"level": str(x["level"])})
#         return dset.cast_column("level", class_label)

#     train_set = convert(df_train)
#     validation_set = convert(df_valid)
#     test_set = convert(df_test)

#     # In thống kê
#     print(f"Train size: {(train_set)}")
#     print(train_set.to_pandas()["level"].value_counts(sort=False), "\n")
    
#     print(f"Validation size: {(validation_set)}")
#     print(validation_set.to_pandas()["level"].value_counts(sort=False), "\n")
    
#     print(f"Test size: {(test_set)}")
#     print(test_set.to_pandas()["level"].value_counts(sort=False))

#     # Bước 6: Lưu dữ liệu về JSON theo định dạng chuẩn (mỗi dòng là 1 object đẹp, KHÔNG phải JSON Lines)
#     def save_to_json(dataset, path):
#         import json
#         df = dataset.to_pandas()
#         records = df.to_dict(orient="records")
#         with open(path, "w", encoding="utf-8") as f:
#             json.dump(records, f, indent=2, ensure_ascii=False)
    
#     output_dir = "/kaggle/working/output_splits"
#     os.makedirs(output_dir, exist_ok=True)
    
#     save_to_json(train_set, f"{output_dir}/train_set.json")
#     save_to_json(validation_set, f"{output_dir}/validation_set.json")
#     save_to_json(test_set, f"{output_dir}/test_set.json")

#     return train_set.remove_columns("level"), validation_set.remove_columns("level"), test_set.remove_columns("level")
# train_set, validation_set, test_set = prepare_train_valid_test_split(hard_dataset_random, dataset)

In [13]:
from datasets import load_dataset
path = '/kaggle/input/stock-dataset-qibot/dataset/finetune_crossEncoder/'
# test_set = load_dataset("json", data_files=f"{path}test_set.json")['train'].remove_columns("level")
# print(test_set)

validation_set = load_dataset("json", data_files=f"{path}validation_set_top.json")['train'].remove_columns("level")
print(validation_set)

train_set = load_dataset("json", data_files=f"{path}train_set_top.json")['train'].remove_columns("level")
print(train_set)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['anchor', 'positive', 'negative'],
    num_rows: 7087
})


Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['anchor', 'positive', 'negative'],
    num_rows: 63779
})


# Building Evaluation set "N-50" for evaluating reranking

In [14]:
# =============================================
# IV. Building Evaluation set "N-50" for evaluating reranking
# =============================================
print(f"{torch.cuda.empty_cache()} {gc.collect()}")
hard_evaluation_set = mine_hard_negatives(
    validation_set,
    embedding_model,
    corpus=validation_set["positive"],  # Use the full dataset as the corpus
    num_negatives=50,  # How many negatives per question-answer pair
    batch_size=128,  # Use a batch size of 4096 for the embedding model
    output_format="n-tuple",  # The output format is (query, positive, negative1, negative2, ...) for the evaluator
    include_positives=True,  # Key: Include the positive answer in the list of negatives
    use_faiss=True,  # Using FAISS is recommended to keep memory usage low (pip install faiss-gpu or pip install faiss-cpu)
)
print(hard_evaluation_set)


None 109
Setting range_max to 54 based on the provided parameters.
Found 7022 unique queries out of 7087 total queries.
Found an average of 1.009 positives per query.


Batches:   0%|          | 0/56 [00:00<?, ?it/s]

Batches:   0%|          | 0/55 [00:00<?, ?it/s]

Querying FAISS index: 100%|██████████| 1/1 [00:00<00:00,  2.51it/s]


Negative candidates mined, preparing dataset...
Metric       Positive       Negative     Difference
Count           7,087        354,350               
Mean           0.7519         0.3827         0.3692
Median         0.7687         0.3412         0.3982
Std            0.1114         0.1384         0.1706
Min           -0.0561         0.1565        -0.8018
25%            0.6807         0.2926         0.2745
50%            0.7687         0.3412         0.3982
75%            0.8383         0.4197         0.4942
Max            0.9642         0.9642         0.7482
Dataset({
    features: ['anchor', 'positive', 'negative_1', 'negative_2', 'negative_3', 'negative_4', 'negative_5', 'negative_6', 'negative_7', 'negative_8', 'negative_9', 'negative_10', 'negative_11', 'negative_12', 'negative_13', 'negative_14', 'negative_15', 'negative_16', 'negative_17', 'negative_18', 'negative_19', 'negative_20', 'negative_21', 'negative_22', 'negative_23', 'negative_24', 'negative_25', 'negative_26', 'neg

In [15]:
hard_evaluation_set[0]

{'anchor': 'Định_nghĩa “ môi_giới chứng_khoán ” theo Luật_Chứng_khoán Việt_Nam là gì ?',
 'positive': 'Môi_giới chứng_khoán là hoạt_động làm trung_gian thực_hiện mua , bán chứng_khoán cho khách_hàng . Công_ty chứng_khoán hoặc cá_nhân hành_nghề môi_giới thay_mặt khách_hàng tìm_kiếm đối_tác , thực_hiện giao_dịch và hưởng phí môi_giới . Đây là một trong các nghiệp_vụ cốt_lõi được quy_định tại Điều 4 và Điều 28 Luật Chứng_khoán .',
 'negative_1': 'Môi_giới chứng_khoán là việc công_ty chứng_khoán làm trung_gian mua , bán chứng_khoán cho khách_hàng trên thị_trường_chứng_khoán . Đây là một trong các nghiệp_vụ kinh_doanh chứng_khoán được pháp_luật quy_định rõ tại Điều 86 Luật Chứng_khoán .',
 'negative_2': 'Môi_giới chứng_khoán là hoạt_động làm trung_gian thực_hiện mua , bán chứng_khoán cho khách_hàng . Công_ty chứng_khoán hoặc cá_nhân hành_nghề môi_giới thay_mặt khách_hàng tìm_kiếm đối_tác , thực_hiện giao_dịch và hưởng phí môi_giới . Đây là một trong các nghiệp_vụ cốt_lõi được quy_định tại Đ

# Define our reranking Evaluators

In [ ]:
# # =============================================
# ## V. Define our reranking Evaluators
# # =============================================
# reranking_evaluator = CrossEncoderRerankingEvaluator(
#     samples=[
#         {
#             "query": sample["anchor"],
#             "positive": [sample["positive"]],
#             "documents": [sample[column_name] for column_name in hard_evaluation_set.column_names[2:]],
#         }
#         for sample in hard_evaluation_set
#     ],
#     batch_size=256,
#     name="triplet",
#     always_rerank_positives=False,
#     mrr_at_k = 10,
#     show_progress_bar= True,
#     write_csv = True
# )

# Evaluation
model AITeamVN/Vietnamese_Reranker

# So sánh với model khác

In [ ]:
# model_vn_reranker = CrossEncoder(
#     "AITeamVN/Vietnamese_Reranker",
#     max_length=512,  # quan trọng
#     tokenizer_args={"truncation": True},
#     num_labels=1
# )

In [ ]:
# print(f"{torch.cuda.empty_cache()} {gc.collect()}")

# print('model_vn_reranker')
# pprint(reranking_evaluator(model_vn_reranker))
# print('model')
# pprint(reranking_evaluator(model))

In [ ]:
# =============================================
## VI. Define the training arguments
# =============================================
# 6a. Define steps
import math
N_data = len(train_set)
batch_size = 8
accumulation = 4
epochs = 14
# 6b. Define args
args = CrossEncoderTrainingArguments(
    # Required parameter:
    output_dir="models/multilingual",
    # Optional training parameters:
    num_train_epochs=epochs,
        
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps = accumulation,
    
    per_device_eval_batch_size=batch_size,
    eval_accumulation_steps=accumulation,
    
    #learning_rate=1e-5, #trước là 2e-5
    
    learning_rate=1e-5,
    lr_scheduler_type="linear",
    
    warmup_ratio=0.1,
    
    warmup_steps=250,# Warmup cho optimizer ổn định hơn
    weight_decay=0.01, #  Giúp mô hình generalize tốt hơn
    fp16=True,  
    bf16=False,
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # losses that use "in-batch negatives" benefit from no duplicates
    
    # Optional tracking/debugging parameters:

    logging_dir="logs",
    logging_strategy="steps",
    logging_steps=100,
    logging_first_step=True,          # ✅ log ngay từ bước đầu tiên
    eval_strategy="epoch",
    #eval_steps=,
    save_strategy="epoch",
    #save_steps=eval_steps,
    save_total_limit=1,

    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
        
    push_to_hub=False,
    hub_model_id=None,
    hub_strategy="every_save",  # hoặc "end", "checkpoint", etc.
    hub_private_repo=True,
    disable_tqdm = False,
    # report_to = [],
    report_to = ["tensorboard"],
    seed = 42,
    run_name="multilingual1",  # Will be used in W&B if `wandb` is installed
)

In [ ]:
# In thông tin các đối tượng
print("--------------------------------Training Dataset:\n", train_set)
print("--------------------------------\nValidation Dataset Sample:\n", validation_set)
# print("--------------------------------\nEvalatuon Dataset Sample:\n", hard_evaluation_set)
print("--------------------------------\CrossEncoderTrainingArguments:\n",args)

In [ ]:
# =============================================
## VII. Define loss function
# =============================================
model = CrossEncoder(
    "sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    #"AITeamVN/Vietnamese_Reranker",
    max_length=512,  # quan trọng
    tokenizer_args={"truncation": True},
    num_labels=1
)

In [ ]:
from sentence_transformers.cross_encoder import losses
from transformers import EarlyStoppingCallback
loss = losses.MultipleNegativesRankingLoss(model,num_negatives = 1)

# Trainer

In [ ]:
print(f"{torch.cuda.empty_cache()} {gc.collect()}")
# multilingual, 2e-5, linear.
# 6. Create the trainer & start training

trainer = CrossEncoderTrainer(
    model=model,
    args=args,
    train_dataset=train_set, # training loss => query, pos, neg
    eval_dataset=validation_set, # validation loss => query, pos, neg
    loss=loss,
    # evaluator = reranking_evaluator # evaluator for reranking task => query, pos, N-neg (N = 50)
)


In [ ]:
%%time
# trainer.train()
import os
checkpoint_path = "/kaggle/input/stock-dataset-qibot/model/checkpoint/checkpoint_14588_multilingual_cross_lr1e-5_batch32"
if os.path.exists(checkpoint_path) and os.path.exists(os.path.join(checkpoint_path, "trainer_state.json")):
    print(f"Checkpoint found at {checkpoint_path}. Resuming training...")
    trainer.train(resume_from_checkpoint=checkpoint_path)

In [ ]:
# pprint(reranking_evaluator(model))

### demo

In [ ]:
# print(f"{torch.cuda.empty_cache()} {gc.collect()}")
# model_ =model_vn_reranker
# train_set_ = train_set.select(range(10))
# validation_set_ = validation_set.select(range(2))
# hard_evaluation_set_ = hard_evaluation_set.select(range(2))
# reranking_evaluator_ = CrossEncoderRerankingEvaluator(
#     samples=[
#         {
#             "query": sample["anchor"],
#             "positive": [sample["positive"]],
#             "documents": [sample[column_name] for column_name in hard_evaluation_set_.column_names[2:]],
#         }
#         for sample in hard_evaluation_set_
#     ],
#     batch_size=4,
#     name="triplet",
#     always_rerank_positives=False,
#     mrr_at_k = 10,
#     show_progress_bar= True,
#     write_csv = True
# )
# loss_ = losses.MultipleNegativesRankingLoss(model_,num_negatives = 1)

# args_ = CrossEncoderTrainingArguments(
#     output_dir="models/multilingual",
#     num_train_epochs=2,
#     per_device_train_batch_size=2,
#     gradient_accumulation_steps = 1,
#     per_device_eval_batch_size=2,
#     per_gpu_train_batch_size = 2,
#     per_gpu_eval_batch_size = 2,
#     eval_accumulation_steps=2,
#     learning_rate=2e-5,
#     lr_scheduler_type="linear",
#     warmup_ratio=0.1,
#     warmup_steps=250,# Warmup cho optimizer ổn định hơn
#     weight_decay=0.01, #  Giúp mô hình generalize tốt hơn
#     fp16=True,  
#     bf16=False,
#     batch_sampler=BatchSamplers.NO_DUPLICATES,  # losses that use "in-batch negatives" benefit from no duplicates
#     logging_dir="logs",
#     logging_strategy="steps",
#     logging_steps=100,
#     logging_first_step=True,          # ✅ log ngay từ bước đầu tiên
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     save_total_limit=2,
#     load_best_model_at_end=True,
#     metric_for_best_model="eval_loss",
#     greater_is_better=True,
#     push_to_hub=False,
#     hub_model_id=None,
#     hub_strategy="every_save",  # hoặc "end", "checkpoint", etc.
#     hub_private_repo=True,
#     disable_tqdm = False,
#     report_to = ["tensorboard"],
#     seed = 42,
#     run_name="multilingual1",  # Will be used in W&B if `wandb` is installed
# )

# trainer_ = CrossEncoderTrainer(
#     model=model_,
#     args=args_,
#     # callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
#     train_dataset=train_set_, # training loss => query, pos, neg
#     eval_dataset=validation_set_, # validation loss => query, pos, neg
#     loss=loss_,
#     #evaluator = reranking_evaluator_ # evaluator for reranking task => query, pos, N-neg (N = 50)
    
# )
# trainer_.train()

# Overview results

In [ ]:
# !pip install tensorboard
# %reload_ext tensorboard
# %tensorboard --logdir logs
# !zip -r logs_tensorboard.zip logs

In [ ]:
# !pip install -q tensorboard
# import matplotlib.pyplot as plt

# from tensorboard.backend.event_processing import event_accumulator

# ea = event_accumulator.EventAccumulator("/kaggle/working/logs/events.out.tfevents.1749202769.19c3ac429ae6.35.1")
# # Reload (i.e., parse) the event file contents
# ea.Reload()
# # Load scalar data
# train_loss = ea.Scalars('train/loss')
# eval_loss = ea.Scalars('eval/loss')

# # Extract step and value
# train_steps = [x.step for x in train_loss]
# train_values = [x.value for x in train_loss]

# eval_steps = [x.step for x in eval_loss]
# eval_values = [x.value for x in eval_loss]

# # Plot both
# plt.figure(figsize=(10, 6))
# plt.plot(train_steps, train_values, label='Train Loss', color='blue')
# plt.plot(eval_steps, eval_values, label='Eval Loss', color='orange')
# plt.xlabel("Step")
# plt.ylabel("Loss")
# plt.title("Train vs Eval Loss")
# plt.legend()
# plt.grid(True)
# plt.show()